##Importing Google Drive for the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.linear_model import LinearRegression
from keras.preprocessing.sequence import pad_sequences


from collections import Counter
import nltk
import string
from nltk.corpus import stopwords

# import regex as re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense
from keras.backend import eval
from tensorflow.keras.optimizers import Adadelta
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,MaxPooling1D
df=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/amazonLabelled.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S          999 non-null    float64
 1   Feedback   999 non-null    object 
 2   Sentiment  999 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.5+ KB


In [ ]:
df.head()

,S,Feedback,Sentiment
0,1.0,"Good case, Excellent value.",Positive
1,2.0,Great for the jawbone.,Positive
2,3.0,Tied to charger for conversations lasting more...,Negative
3,4.0,The mic is great.,Positive
4,5.0,I have to jiggle the plug to get it to line up...,Negative


In [ ]:
from nltk.corpus.reader import reviews
reviews = df['Feedback']
label = df['Sentiment']

In [ ]:
import string
from nltk.corpus import stopwords
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop = set(stopwords.words('english'))

Stopwords: Those words which are not meant to be used to describe the sentiment of the feedback. NLTK contains the list of 40 stop-words by default, including "a", "an", "the", "at", "of", etc.

In [ ]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)

    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')

    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])

    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])

    doco_clean = ([p.sub("", x).strip() for x in doco_clean])

    return doco_clean

In [ ]:
review_cleans = [clean_document(doc) for doc in reviews];
sentences = [' '.join(r) for r in review_cleans ]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [ ]:
text_sequences = np.array(tokenizer.texts_to_sequences(sentences))
sequence_dict = tokenizer.word_index
word_dict = dict((num, val) for (val, num) in sequence_dict.items())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
print(text_sequences)

[list([3, 17, 18, 178]) list([2, 310])
 list([674, 31, 311, 422, 148, 675, 71]) list([240, 2])
 list([676, 84, 25, 179, 72, 25, 241, 73])
 list([128, 677, 128, 678, 312, 679, 680, 242, 11, 11])
 list([180, 681, 243]) list([682, 129, 423, 32]) list([49, 32, 19])
 list([9, 5, 2]) list([94, 149, 181, 8, 424, 8])
 list([50, 683, 684, 182, 425, 685, 313, 686, 9, 6]) list([3, 5, 314])
 list([80, 687, 13, 244, 42]) list([95, 21, 11, 245, 426, 1])
 list([427, 315, 688]) list([58, 3]) list([7, 2])
 list([689, 246, 150, 81, 690, 59, 691, 14, 109])
 list([130, 60, 428, 692, 693, 45, 25, 316])
 list([43, 12, 694, 429, 247, 695]) list([696, 697])
 list([317, 430, 28, 8, 50, 248, 151, 96, 110, 183])
 list([43, 698, 61, 8]) list([2, 318, 1, 699])
 list([431, 1, 97, 129, 23, 184, 1]) list([85, 249, 319, 185])
 list([86, 700, 74, 111, 432, 701, 111, 1]) list([186, 51])
 list([152, 112, 1, 12, 702, 703]) list([4, 704, 86, 16, 705, 153, 706])
 list([433, 707, 38, 708, 709]) list([50, 98, 250, 710, 251, 1

In [ ]:
print(sequence_dict)

{'phone': 1, 'great': 2, 'good': 3, 'product': 4, 'quality': 5, 'headset': 6, 'works': 7, 'battery': 8, 'sound': 9, 'well': 10, 'one': 11, 'use': 12, 'ear': 13, 'would': 14, 'work': 15, 'like': 16, 'case': 17, 'excellent': 18, 'time': 19, 'price': 20, 'recommend': 21, 'really': 22, 'best': 23, 'service': 24, 'get': 25, 'nice': 26, 'also': 27, 'new': 28, 'love': 29, 'worked': 30, 'charger': 31, 'money': 32, 'first': 33, 'buy': 34, 'item': 35, 'better': 36, 'ever': 37, 'car': 38, 'bluetooth': 39, 'even': 40, 'easy': 41, 'comfortable': 42, 'bought': 43, 'reception': 44, 'could': 45, 'used': 46, 'poor': 47, 'happy': 48, 'waste': 49, 'two': 50, 'charge': 51, 'made': 52, 'still': 53, 'bad': 54, 'purchase': 55, 'cell': 56, 'worst': 57, 'far': 58, 'long': 59, 'motorola': 60, 'problem': 61, 'life': 62, 'fine': 63, 'camera': 64, 'calls': 65, 'enough': 66, 'thing': 67, 'device': 68, 'piece': 69, 'got': 70, 'problems': 71, 'right': 72, 'volume': 73, 'hear': 74, 'make': 75, 'clear': 76, 'much': 77,

In [ ]:
reviews_encoded = [];
for i,review in enumerate(review_cleans):
    reviews_encoded.append([sequence_dict[x] for x in review]);

In [ ]:
print(reviews_encoded[135])

[3, 490, 20]


In [ ]:
max_cap =8;
X = pad_sequences(reviews_encoded, maxlen=max_cap, truncating='post')
Y = np.array([[0,1] if '0' in label else [1,0] for label in label])

In [ ]:
np.random.seed(1024);
random_posits = np.arange(len(X))
np.random.shuffle(random_posits);

In [ ]:
X = X[random_posits];
Y = Y[random_posits];
train_cap = int(0.85 * len(X));
dev_cap = int(0.93 * len(X));

In [ ]:
X_train, Y_train = X[:train_cap], Y[:train_cap]
X_dev, Y_dev = X[train_cap:dev_cap], Y[train_cap:dev_cap]
X_test1, Y_test1 = X[dev_cap:], Y[dev_cap:]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import eval
from keras.layers import LSTM
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

model1 = Sequential();
model1.add(Embedding(len(word_dict)+1, max_cap, input_length=max_cap));
#adding a LSTM layer of dim 1--
model1.add(LSTM(150, return_sequences=True));
model1.add(LSTM(150, return_sequences=False));
#adding a dense layer with activation function of relu
model1.add(Dense(100, activation='relu'));#best 50,relu
#adding the final output activation with activation function of softmax
model1.add(Dense(2, activation='sigmoid'));
print(model1.summary());
optimizer = Adadelta(lr=0.001, decay=2);

model1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 8, 8)              13440     
                                                                 
 lstm_50 (LSTM)              (None, 8, 150)            95400     
                                                                 
 lstm_51 (LSTM)              (None, 150)               180600    
                                                                 
 dense_50 (Dense)            (None, 100)               15100     
                                                                 
 dense_51 (Dense)            (None, 2)                 202       
                                                                 
Total params: 304,742
Trainable params: 304,742
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adadelta.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adadelta, self).__init__(name, **kwargs)


In [ ]:
model1.fit(X_train, Y_train, batch_size=16, epochs=5, validation_data=(X_dev, Y_dev))

Epoch 1/5
54/54 [==============================] - 7s 55ms/step - loss: 0.6921 - accuracy: 0.8681 - val_loss: 0.6921 - val_accuracy: 0.8875
Epoch 2/5
54/54 [==============================] - 2s 33ms/step - loss: 0.6921 - accuracy: 0.8693 - val_loss: 0.6921 - val_accuracy: 0.8875
Epoch 3/5
54/54 [==============================] - 2s 34ms/step - loss: 0.6921 - accuracy: 0.8716 - val_loss: 0.6921 - val_accuracy: 0.8875
Epoch 4/5
54/54 [==============================] - 2s 33ms/step - loss: 0.6921 - accuracy: 0.8728 - val_loss: 0.6921 - val_accuracy: 0.8875
Epoch 5/5
54/54 [==============================] - 2s 33ms/step - loss: 0.6921 - accuracy: 0.8740 - val_loss: 0.6921 - val_accuracy: 0.8875
